# Lecture 6 yex8wb

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [5]:
df = pd.read_csv("tech_stocks.csv")
df.head()

,Unnamed: 0,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
0,0,MSFT,2016-05-16,50.80,51.96,50.75,51.83,49.7013,20032017,1.0
1,1,MSFT,2002-01-16,68.85,69.84,67.85,67.87,22.5902,30977700,1.0
2,2,MSFT,2001-09-18,53.41,55.00,53.17,54.32,18.0802,41591300,1.0
3,3,MSFT,2007-10-26,36.01,36.03,34.56,35.03,27.2232,288121200,1.0
4,4,MSFT,2014-06-27,41.61,42.29,41.51,42.25,38.6773,74640000,1.0


In [8]:
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["symbol", "date"]).reset_index(drop=True)
df.head()

,Unnamed: 0,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
0,19460361,AAPL,1998-01-02,13.63,16.25,13.50,16.25,0.5103,6411700,1.0
1,19459978,AAPL,1998-01-05,16.50,16.56,15.19,15.88,0.4987,5820300,1.0
2,19460471,AAPL,1998-01-06,15.94,20.00,14.75,18.94,0.5948,16182800,1.0
3,19460562,AAPL,1998-01-07,18.81,19.00,17.31,17.50,0.5496,9300200,1.0
4,19460060,AAPL,1998-01-08,17.44,18.62,16.94,18.19,0.5713,6910900,1.0


In [ ]:
# stock go up or down
df["ret"] = (df["close"] - df["open"]) / df["open"]

#How volatile the stock was that day
df["range"] = (df["high"] - df["low"]) / df["open"]

#each stock seperate
df["close_next"] = df.groupby("symbol")["close"].shift(-1)

#0/1 target variable
df["up_next"] = (df["close_next"] > df["close"]).astype(int)
df.head()


,Unnamed: 0,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient,close_next,up_next,ret,range
0,19460361,AAPL,1998-01-02,13.63,16.25,13.50,16.25,0.5103,6411700,1.0,15.88,0,0.192223,0.201761
1,19459978,AAPL,1998-01-05,16.50,16.56,15.19,15.88,0.4987,5820300,1.0,18.94,1,-0.037576,0.083030
2,19460471,AAPL,1998-01-06,15.94,20.00,14.75,18.94,0.5948,16182800,1.0,17.50,0,0.188206,0.329360
3,19460562,AAPL,1998-01-07,18.81,19.00,17.31,17.50,0.5496,9300200,1.0,18.19,1,-0.069644,0.089846
4,19460060,AAPL,1998-01-08,17.44,18.62,16.94,18.19,0.5713,6910900,1.0,18.19,0,0.043005,0.096330


In [25]:
#no missing values
df = df.dropna(subset=["ret", "range", "up_next"]).copy()

#distance calc
def minmax(x):
    return (x - x.min()) / (x.max() - x.min())

X = df[["ret", "range"]].apply(minmax)
y = df["up_next"]


In [ ]:
# ASK PROF T about test_size (Change to .2, .5, .7, (1-p so technically 1-.2=.8 for train, training vs testing tradeoff))
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=100, stratify=y
)

#create k value 
k_grid = np.arange(1, 80, 2)  
test_acc = []
train_acc = []

for k in k_grid:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    train_acc.append(model.score(X_train, y_train))
    test_acc.append(model.score(X_test, y_test))

#indec w largest test acc
k_best = k_grid[np.argmax(test_acc)]
print("Goated k:", k_best, "Goated test accuracy:", max(test_acc))


# REffitting 
best_model = KNeighborsClassifier(n_neighbors=k_best).fit(X_train, y_train)
y_hat = best_model.predict(X_test)

print("\nConfusion matrix:\n", confusion_matrix(y_test, y_hat))
print("Accuracy:", accuracy_score(y_test, y_hat))

Goated k: 19 Goated test accuracy: 0.5182311869666408

Confusion matrix:
 [[1498 1636]
 [1469 1842]]
Accuracy: 0.5182311869666408
